In [40]:
from code_analysis import CFG, CFGReader
import queue
import time
import numpy as np

In [41]:
##get the nodes for f-open and f-close
reader = CFGReader()
file = "../tp4/part_1/test.php.cfg.json"
graph = reader.read_cfg(file)


In [42]:

def defs(node : int, cfg : CFG) :
    if cfg.get_type(node) == "Variable" :
        #get the parent node
        #look for all BinOP nodes
        child = cfg.get_children(node)[0]
        if cfg.get_type(child) == "BinOP" and cfg.get_image(child) == "=" :
            return set([node])
    return set()

            
def kill_def(node : int, cfg : CFG) :
    defs_nodes = defs(node, cfg)
    if len(defs_nodes) == 0:
        return set()
    to_return = set()
    names_var_defs = set()
    for n in defs_nodes :
        names_var_defs.add(cfg.get_image(n))
    for n in cfg.get_node_ids() :
        if cfg.get_type(n) == "Variable" and cfg.get_image(n) in names_var_defs and n in defs(n,cfg) :
            to_return.add(n)
    return to_return

def refs(node : int, cfg : CFG) :
    if cfg.get_type(node) == "Variable" :
        return set([node]).difference(defs(node, cfg))
    return set()

def kill_ref(node : int, cfg : CFG) :
    
    if cfg.get_type(node) == "Variable" :
        defs_node = defs(node, cfg)
        var_name = cfg.get_image(node)
        node_with_same_var_name = set()
        for node_id in cfg.get_node_ids() :
            if cfg.get_type(node_id) == 'Variable' and cfg.get_image(node_id) == var_name :
                node_with_same_var_name.add(node_id)
        return defs_node.intersection(node_with_same_var_name)
            
    return set()
def possibly_reaching_definition(node_set : set, cfg : CFG) :
    IN : dict[set] = {}
    OUT : dict[set] = {}
    GEN : dict[set] = {}
    KILL : dict[set] = {}
    for node in cfg.get_node_ids() :
        IN[node] = set()
        OUT[node] = set()
        GEN[node] = set()
        KILL[node] = set()
    for node in node_set :
            
        GEN[node] = defs(node, cfg)
        KILL[node] = kill_def(node, cfg)
        
    changes = True
    while changes :
        changes = False
        for node in cfg.get_node_ids() :
            predecessors = cfg.get_parents(node)
            for pred in predecessors :
                IN[node] = IN[node].union(OUT[pred])
            old_out = OUT[node].copy()
            OUT[node] = GEN[node].union(IN[node] - KILL[node])
            if OUT[node] != old_out :
                changes = True
    print("end")
    return IN, OUT, GEN, KILL

 
def possibly_reachable_reference(node_set : set, cfg : CFG) :
    IN : dict[set] = {}
    OUT : dict[set] = {}
    GEN : dict[set] = {}
    KILL : dict[set] = {}
    for node in cfg.get_node_ids() :
        IN[node] = set()
        OUT[node] = set()
        GEN[node] = set()
        KILL[node] = set()
    for node in node_set :
        GEN[node] = set(refs(node, cfg))
        KILL[node] = set(kill_ref(node, cfg))
        
    changes = True
    while changes :
        changes = False
        for node in cfg.get_node_ids() :
            successors = cfg.get_children(node)
            for suc in successors :
                OUT[node] = OUT[node].union(IN[suc])
            OLD_IN = IN[node].copy()
            IN[node] = GEN[node].union(OUT[node] - KILL[node])
            if IN[node] != OLD_IN :
                changes = True
    print("end")
    return IN, OUT, GEN, KILL
    
            
            

### Verifications Pairs/Définitions Definitions/Pairs

In [43]:
def get_all_refs(cfg : CFG) :
    ref = set()
    for node in cfg.get_node_ids() :
        ref = ref.union(refs(node, cfg))
    return ref

def get_all_defs(cfg : CFG) :
    all_def = set()
    for node in cfg.get_node_ids() :
        all_def = all_def.union(defs(node, cfg))
    return all_def

In [45]:
def extract_def_ref_pairs(cfg : CFG):
    # Étape 1 et 2 : Déterminer les références et leurs définitions correspondantes,
    # ainsi que les définitions et leurs références correspondantes
    all_refs = get_all_refs(cfg)
    all_defs = get_all_defs(cfg)
    
            
    defs_in, OUT, _, _ = possibly_reaching_definition(cfg.get_node_ids(), cfg)
    defs_of_refs = {i : defs_in[i] for i in all_refs}
    ref_in, _, _, _ = possibly_reachable_reference(cfg.get_node_ids(Ò), cfg)
    refs_of_defs = {i : ref_in[i] for i in all_defs}
    

    # Étape 3 : Vérifier que les ensembles de paires définitions/références
    # et références/définitions correspondent
    for node in cfg.get_node_ids():
        if cfg.get_type(node) == "Variable":
            if node in ref_in:
                assert set(defs_in[node]) == set(ref_in[node]), f"Incohérence pour la référence {node}"
            else:
                assert len(defs_in[node]) == 0, f"Incohérence pour la référence {node}"

            if node in defs(node, cfg):
                assert set(OUT[node]) == set(refs(node, cfg)), f"Incohérence pour la définition {node}"

    # Étape 4 : Extraire les ensembles de paires définitions/références
    # et références/définitions
    def_ref_pairs = set()
    ref_def_pairs = set()

    for node in cfg.get_node_ids():
        if cfg.get_type(node) == "Variable":
            if node in ref_in:
                for def_node in ref_in[node]:
                    ref_def_pairs.add((node, def_node))

            if node in defs(node, cfg):
                for ref_node in refs(node, cfg):
                    def_ref_pairs.add((node, ref_node))

    return def_ref_pairs, ref_def_pairs

# Utilisation
cfg = graph
def_ref_pairs, ref_def_pairs = extract_def_ref_pairs(cfg)

print("Paires définitions/références :")
for pair in def_ref_pairs:
    print(f"Définition : {pair[0]}, Référence : {pair[1]}")

print("\nPaires références/définitions :")
for pair in ref_def_pairs:
    print(f"Référence : {pair[0]}, Définition : {pair[1]}")
    

end
end
Paires définitions/références :

Paires références/définitions :
